## Imports of utility functions we will be using
below we import the work for our residual network and deep hybrid scatter network

In [ ]:
import sys
import os
sys.path.append('/home/ubuntu/feature_viz/resnet')
sys.path.append('/home/ubuntu/feature_viz/ScatteringTransform/src/model')
from flags import define_flags as scatternet_define_flags
from train_mnist import train_model as scatternet_train
from res_features import train as resnet_train
from visualize import visualize_features
import sklearn
import tensorflow as tf

mnist = tf.contrib.learn.datasets.load_dataset("mnist")

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


# FEATURE EXTRACTION WITH RESIDUAL NETWORK AND FINE TUNING
## Resnet 50 pretrained from the Kth layer and higher
* pretraining Icslvr 2017
* blocks before k are frozen
* classification is done w/ 3 fully connected layers. The first 2 use rectified linear units as acitvation functions while the final layer has a softmax
* features are extracted from the 4th residual block. In the 50 layer residual network this is the block immediately before generating logits for classification.

In [ ]:
resnet_classifier = resnet_train(freeze_before_k=3) # all blocks before block k are frozen for training

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from /home/ubuntu/feature_viz/resnet/resnet_v1_50.ckpt
iter 0 train accuracy: 0.125


# FEATURE EXTRACTION WITH SCATTER NETWORK
## Scatternet Training and Testing on MNIST
The work for scatternet was mostly based off the work done by [tdeboissiere](https://github.com/tdeboissiere) found [here](https://github.com/tdeboissiere/DeepLearningImplementations/tree/master/ScatteringTransform). This scatter network is actually a bit different than the scatter network found in Brunna & Mallat's work. What we analyze here is  a deep hybrid scatter network. We use scatter transforms similar to those found in [Brunna & Mallat's work](https://arxiv.org/abs/1203.1513) followed by a few convolutional layers and fully connected layers. A detailed explanation of how this works can be found in Oyallon's [Deep Hybrid Networks paper](https://arxiv.org/abs/1703.08961).

In [ ]:
scatternet_define_flags() 
scatternet_classifier = scatternet_train()  # no pretrained weights for the hybrid scatter net so we do not need to specify a k

# FEATURE VISUALIZATION
Sample some data points from MNIST to map to a feature space with both of our classifiers. Once we have both of these new feature spaces we can use TSNE to reduce dimensionality to a space easily visualized.

In [ ]:
x_viz, y_viz = mnist.test.next_batch(1280)

## TSNE on 50 layer residual network features
Looks like we are forming some clusters of each of the classes however there is heavy overlap. For example 2, 3, 5, and 8 all have clusters that that appear in roughly the same space.

In [ ]:
resnet_features_viz = resnet_classifier.get_features(x_viz)
visualize_features(resnet_features_viz, y_viz, 'resnet50')

## TSNE on Deep Hybrid Network
Class separation is much stronger here than we experienced in our 50 layer residual network. I am guessing this is because the Deep Hybrid Network allowed for all the weights to be trained. While we froze all the weights and biases below the 3rd block of the resnet.

In [ ]:
scatternet_features_viz = scatternet_classifier(x_viz)
visualize_features(scatternet_features_viz, y_viz, 'scatternet')

# IMAGAGE CLASSIFICATION COMPARISON
## Multilayer Neural Network vs. Logistic Regression vs. Support Vector Machine

In [ ]:
def logistic_regression(train_features, train_labels, test_features, test_labels):
    lm = sklearn.linear_model.Logistic_Regression(multi_class='multinomial')
    lm.fit(train_features, train_labels)
    score = lm.score(test_features, test_labels)
    return score

def support_vector_machine(train_features, train_labels, test_features, test_labels):
    svm = sklearn.svm.SVC()
    svm.fit(train_features, train_labels)
    score = svm.score(test_features, test_labels)
    return score

In [ ]:
x_train, y_train = mnist.train.next_batch(1280)
scatternet_features_train = scatternet_classifier.get_features(x_train)
resnet_features_train = resnet_classifier.get_features(x_train)

lm_res_score = logistic_regression(resnet_features, y_train, resnet_features_viz, y_viz)
svm_res_score = support_vector_machine(resnet_features, y_train, resnet_features_viz, y_viz)

lm_scatternet_score = logistic_regression(scatternet_features, y_train, scatternet_features_viz, y_viz)
svm_scatternet_score = support_vector_machine(scatternet_features, y_train, scatternet_features_viz, y_viz)